In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import import_ipynb
import tensorflow as tf
from scipy import misc
import cv2
import numpy as np
import facenet
import detect_face
import os
import time
import pickle

importing Jupyter notebook from facenet.ipynb
importing Jupyter notebook from detect_face.ipynb


In [2]:
#input_video="akshay_mov.mp4"
modeldir = './model/20170511-185253.pb'
classifier_filename = './class/classifier.pkl'
npy='./npy'
train_img="./train_img"

In [5]:
# Using OpenCV to initialize the webcam
cap = cv2.VideoCapture(0)

with tf.Graph().as_default():
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.6)
    sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=False))
    with sess.as_default():
        pnet, rnet, onet = detect_face.create_mtcnn(sess, npy)

        minsize = 20  # minimum size of face
        threshold = [0.6, 0.7, 0.7]  # three steps's threshold
        factor = 0.709  # scale factor
        margin = 44
        frame_interval = 3
        batch_size = 1000
        image_size = 182
        input_image_size = 160
        
        HumanNames = os.listdir(train_img)
        HumanNames.sort()

        print('Loading Modal')
        facenet.load_model(modeldir)
        images_placeholder = tf.get_default_graph().get_tensor_by_name("input:0")
        embeddings = tf.get_default_graph().get_tensor_by_name("embeddings:0")
        phase_train_placeholder = tf.get_default_graph().get_tensor_by_name("phase_train:0")
        embedding_size = embeddings.get_shape()[1]


        classifier_filename_exp = os.path.expanduser(classifier_filename)
        with open(classifier_filename_exp, 'rb') as infile:
            (model, class_names) = pickle.load(infile)

        #video_capture = cv2.VideoCapture(input_video)
        c = 0


        print('Start Recognition')
        prevTime = 0
        while True:
            ret, frame = cap.read()

            #frame = cv2.resize(frame, (0,0), fx=0.8, fy=0.8)    #resize frame (optional)

            curTime = time.time()+1    # calc fps
            timeF = frame_interval

            if (c % timeF == 0):
                find_results = []

                if frame.ndim == 2:
                    frame = facenet.to_rgb(frame)
                frame = frame[:, :, 0:3]
                bounding_boxes, _ = detect_face.detect_face(frame, minsize, pnet, rnet, onet, threshold, factor)
                nrof_faces = bounding_boxes.shape[0]
                print('Detected_FaceNum: %d' % nrof_faces)

                if nrof_faces > 0:
                    det = bounding_boxes[:, 0:4]
                    img_size = np.asarray(frame.shape)[0:2]

                    cropped = []
                    scaled = []
                    scaled_reshape = []
                    bb = np.zeros((nrof_faces,4), dtype=np.int32)

                    for i in range(nrof_faces):
                        emb_array = np.zeros((1, embedding_size))

                        bb[i][0] = det[i][0]
                        bb[i][1] = det[i][1]
                        bb[i][2] = det[i][2]
                        bb[i][3] = det[i][3]

                        # inner exception
                        if bb[i][0] <= 0 or bb[i][1] <= 0 or bb[i][2] >= len(frame[0]) or bb[i][3] >= len(frame):
                            print('Face is very close!')
                            continue

                        cropped.append(frame[bb[i][1]:bb[i][3], bb[i][0]:bb[i][2], :])
                        cropped[i] = facenet.flip(cropped[i], False)
                        scaled.append(misc.imresize(cropped[i], (image_size, image_size), interp='bilinear'))
                        scaled[i] = cv2.resize(scaled[i], (input_image_size,input_image_size),
                                               interpolation=cv2.INTER_CUBIC)
                        scaled[i] = facenet.prewhiten(scaled[i])
                        scaled_reshape.append(scaled[i].reshape(-1,input_image_size,input_image_size,3))
                        feed_dict = {images_placeholder: scaled_reshape[i], phase_train_placeholder: False}
                        emb_array[0, :] = sess.run(embeddings, feed_dict=feed_dict)
                        predictions = model.predict_proba(emb_array)
                        print(predictions)
                        best_class_indices = np.argmax(predictions, axis=1)
                        best_class_probabilities = predictions[np.arange(len(best_class_indices)), best_class_indices]
                        # print("predictions")
                        print(best_class_indices,' with accuracy ',best_class_probabilities)

                        # print(best_class_probabilities)
                        if best_class_probabilities>0.53:
                            cv2.rectangle(frame, (bb[i][0], bb[i][1]), (bb[i][2], bb[i][3]), (0, 255, 0), 2)    #boxing face

                            #plot result idx under box
                            text_x = bb[i][0]
                            text_y = bb[i][3] + 20
                            print('Result Indices: ', best_class_indices[0])
                            print(HumanNames)
                            for H_i in HumanNames:
                                if HumanNames[best_class_indices[0]] == H_i:
                                    result_names = HumanNames[best_class_indices[0]]
                                    result_accuracy = str(best_class_probabilities[0])
                                    result = result_names + result_accuracy
                                    cv2.putText(frame, result, (text_x, text_y), cv2.FONT_HERSHEY_COMPLEX_SMALL,
                                                .5, (0, 0, 255), thickness=1, lineType=1)
                else:
                    print('Alignment Failure')
            # c+=1
            cv2.imshow('Video', frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()

Loading Modal
Model filename: ./model/20170511-185253.pb
Start Recognition
Detected_FaceNum: 0
Alignment Failure
Detected_FaceNum: 0
Alignment Failure
Detected_FaceNum: 0
Alignment Failure
Detected_FaceNum: 0
Alignment Failure
Detected_FaceNum: 0
Alignment Failure
Detected_FaceNum: 0
Alignment Failure
Detected_FaceNum: 0
Alignment Failure
Detected_FaceNum: 0
Alignment Failure
Detected_FaceNum: 0
Alignment Failure
Detected_FaceNum: 0
Alignment Failure
Detected_FaceNum: 0
Alignment Failure
Detected_FaceNum: 0
Alignment Failure
Detected_FaceNum: 1


/home/yasir/.local/lib/python3.6/site-packages/ipykernel_launcher.py:82: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


[[0.01619775 0.12405306 0.51406875 0.15191573 0.13645793 0.05730677]]
[2]  with accuracy  [0.51406875]
Detected_FaceNum: 1
[[0.03939087 0.05538691 0.22744627 0.21966828 0.35023902 0.10786865]]
[4]  with accuracy  [0.35023902]


/home/yasir/.local/lib/python3.6/site-packages/ipykernel_launcher.py:82: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


Detected_FaceNum: 1
[[0.0341723  0.11212036 0.43195618 0.08721623 0.20754492 0.12699002]]
[2]  with accuracy  [0.43195618]


...

/home/yasir/.local/lib/python3.6/site-packages/ipykernel_launcher.py:82: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


Detected_FaceNum: 1
[[0.00424742 0.16856381 0.36054297 0.10847268 0.21039156 0.14778157]]
[2]  with accuracy  [0.36054297]


/home/yasir/.local/lib/python3.6/site-packages/ipykernel_launcher.py:82: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


Detected_FaceNum: 1
[[0.00520823 0.06330628 0.44186062 0.05235446 0.30526389 0.13200651]]
[2]  with accuracy  [0.44186062]


/home/yasir/.local/lib/python3.6/site-packages/ipykernel_launcher.py:82: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
